## Importation librairie

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os
from google.colab import files
from google.colab import drive
import numpy as np

## Importation fichier

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Chemin du dossier dans votre Google Drive
chemin_dossier = '/content/drive/My Drive/Donnée_ABACUS'

# Liste des fichiers dans le dossier
contenu_dossier = os.listdir(chemin_dossier)

# Importation des données

## Définition des colonnes à garder

In [ ]:
liste_noms_colonnes = ['Origin of message',
 'Flight date',
 'Sequence number',
 'Time of departure/entry',
 'Departure aerodrome',
 'Arrival aerodrome',
 'Type of aircraft',
 'Flight identification',
 #'Original Flight Identification',
 'Aircraft Registration',
 #'Aircraft Registration in FPL',
 'User ICAO Code',
 'User Number',
 'VAT Number',
 'User Nationality',
 'Correction Code',
 'IFPLID',
 '24-bit Aircraft Address',
 'Source of the Aircraft Address',
 'Main Exemption Code',
 #''1st Supp. Ex. Code',
 #''2nd Supp. Ex. Code',
 #''3rd Supp. Ex. Code',
 'Flight Message ID',
 #''Claim Number',
 'Pro forma number',
 'Pro forma line number',
 'Bill or Credit Note Reference',
 'MTOW',
 'Weight Coefficient',
 'Distance Coefficient',
 'Service Units',
 'National Route Charge in BZ',
 'Admin. Route Charge in BZ',
 'National Route Charge for SP',
 'Admin. Route Charge for SP',
 'Exemption Indicator',
 #'Exemption Code',
 'Flight category',
# 'VAT Doc. Ref.',
 #'Original VAT Doc. Ref',
 'VAT Rate',
 #'VAT Code',
 'VAT on National Route Charge for SP',
 'VAT on Admin. Route Charge for SP']
# Colonne que l'on souhaite sauvegardé :
noms_colonnes = ['Origin of message',
 'Flight date',
 'Time of departure/entry',
 'Departure aerodrome',
 'Arrival aerodrome',
 'Type of aircraft',
 'User Number',
 'User Nationality',
 'Correction Code',
 'Main Exemption Code',
 'Weight Coefficient',
 'Distance Coefficient',
 'Service Units',
 'Exemption Indicator',
 'Flight category',]

## Définition de fonction transformant les formats des données

In [ ]:
def convertir_format(data):
  data['Flight date'] = pd.to_datetime(data['Flight date'], format='%y%m%d')
  data['Weight Coefficient'] = data['Weight Coefficient'].str.replace(',', '.').str.lstrip('0')
  data['Distance Coefficient'] = data['Distance Coefficient'].str.replace(',', '.').str.lstrip('0')
  data['Service Units'] = data['Service Units'].str.replace(',', '.').str.lstrip('0')


In [ ]:
def convertir_dtype_category(data):
  data["Origin of message"] = data["Origin of message"].astype("category")
  data['Departure aerodrome'] = data['Departure aerodrome'].astype("category")
  data['Arrival aerodrome'] = data['Arrival aerodrome'].astype("category")
  data['Type of aircraft'] = data['Type of aircraft'].astype("category")
  data['User Nationality'] = data['User Nationality'].astype("category")
  data['Main Exemption Code'] = data['Main Exemption Code'].astype("category")
  data['Correction Code'] = data['Correction Code'].astype("category")
  data['Exemption Indicator'] = data['Exemption Indicator'].astype("category")
  data['Flight category'] = data['Flight category'].astype("category")
  data['User Number'] = data['User Number'].astype("category")

In [ ]:
def convertir_dtype_numeric(data):
  data['Weight Coefficient'] = pd.to_numeric(data['Weight Coefficient'], errors='coerce')
  data['Distance Coefficient'] = pd.to_numeric(data['Distance Coefficient'], errors='coerce')
  data['Service Units'] = pd.to_numeric(data['Service Units'], errors='coerce')

In [ ]:
# Fonction pour extraire la partie désirée du code OACI avec les nouvelles spécifications
def extract_oaci_code(code):
    if len(code) >= 2:
        if code[0] == 'L':
            if code[:2] in ['LA', 'LB', 'LC', 'LD', 'LH', 'LJ', 'LL', 'LM', 'LR', 'LT', 'LU', 'LX', 'LY', 'LZ', 'LG', 'LK', 'LO', 'LN', 'LW', 'LQ']:
                return code[0]
            elif code[:2] == 'LF':
                return code
            else:
                return code[:2]
        elif code[0] == 'E':
            if code[:2] in ['EV', 'EY', 'ES', 'EP', 'EN', 'EL', 'EK', 'EH', 'EF', 'EE', 'ET']:
                return code[0]
            else:
                return code[:2]
        elif code[0] == 'N':
            if code[:2] == 'NL' or code[:2] == 'NT':
                return code[:2]
            else:
                return code[0]
        elif code[0] in ['W', 'A', 'P', 'Y', 'N']:
            return 'Y'
        elif code[0] in ['M', 'T']:
            return 'M'
        elif code[0] in ['H', 'F']:
            return 'H'
        elif code[0] in ['Z', 'R', 'V']:
            return 'Z'
        elif code[0] in ['C', 'B']:
            return 'C'
        else:
            return code[0]
    else:
        return code

## Importation des données

In [ ]:
# Créer un DataFrame vide
df = pd.DataFrame()

# Parcourir les fichiers dans le répertoire
for filename in contenu_dossier:
    if filename.endswith('.TXT'):  # Vérifier si c'est un fichier TXT
        # Charger le fichier dans une DataFrame
        dfs = pd.read_csv(os.path.join(chemin_dossier, filename), sep=';', usecols = noms_colonnes)

        # Convertir sous les bon formats (gain de place)
        # Appliquer la fonction à vos colonnes
        #dfs['Departure aerodrome'] = dfs['Departure aerodrome'].apply(extract_oaci_code)
        #dfs['Arrival aerodrome'] = dfs['Arrival aerodrome'].apply(extract_oaci_code)
        convertir_format(dfs)
        convertir_dtype_numeric(dfs)


        #if filename == "LF20LF10%S%ABACUS_FLSPBZ03_2002.TXT":
        #  dfs = dfs[dfs["Flight date"] >= '2020-01-01']
        # Ajouter la DataFrame à la liste
        df = pd.concat([df, dfs], ignore_index= True)
        del dfs

convertir_dtype_category(df)


In [ ]:
df = df[df['Flight date'] >= '2017-04-01']

In [ ]:
df = df.drop_duplicates(keep = 'first', subset= ['Origin of message', 	'Flight date', 	'Time of departure/entry', 	'Departure aerodrome', 	'Arrival aerodrome', 	'Type of aircraft', 	'User Number'])
df['Flight date'] = pd.to_datetime(df['Flight date'])
#df.set_index('Flight date', inplace=True)

## Filtrage

In [ ]:
data = df.copy()

# Filtrage

In [ ]:
data = data[data['Origin of message'] == 'LF']

In [ ]:
data = data[~data['Departure aerodrome'].str.startswith('LF')]

In [ ]:
dataslice = data.copy()

In [ ]:
dataslice['Departure aerodrome'] = dataslice['Departure aerodrome'].str.slice(0, 2)
dataslice = dataslice[dataslice['Flight date'] >= '2022-04-01']

In [ ]:
dataslice['Departure aerodrome'].value_counts()

,count
Departure aerodrome,
DA,13179
FA,6649
DT,3638
DN,3594
HK,2152
FK,1534
GM,1389
DG,1368
FZ,1091


In [ ]:
data2 = data[data['Flight date'] >= '2022-04-01']

In [ ]:
data2['Departure aerodrome'].value_counts()

,count
Departure aerodrome,
FAOR,4117
DAAG,3738
DABC,3378
DTTJ,2451
FACT,2429
DNMM,2421
HKJK,2139
DAAE,1956
DABB,1412


In [ ]:
donnes_50t_AFR_TRA = data[(data["User Number"].isin([227, 991, 21680, 20319, 26626, 29910, 2723, 32673, 35579])) & (df["MTOW"] <= '0050,0')]

In [ ]:
# Exporter le DataFrame en format texte (txt)
donnes_50t_AFR_TRA.to_csv('donnes_50t_AFR_TRA.txt', sep='\t', index=False)

# Télécharger le fichier texte
files.download('donnes_50t_AFR_TRA.txt')

# Exporter le DataFrame en format Excel
donnes_50t_AFR_TRA.to_excel('donnes_50t_AFR_TRA.xlsx', index=False)

# Télécharger le fichier Excel
files.download('donnes_50t_AFR_TRA.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>